In [ ]:
# AIMS OF THE PROJECT:
# - Top Stock Mentions Today (Tickers), Last 2 days, 5 days, 7 Days, Custom number. Provide it in the list or barchart
# - Print out Comments that mentioned these tickers or just for one ticker
# - Provide visualisation in form of a graph
# - Create a Jupyter Notebooko tool with buttons that allows non coder to do such tasks.


In [ ]:
from datetime import datetime, date, time
import re

import pandas as pd
import numpy as np

import json
import praw
import requests

import jupyter

In [ ]:
# Jupyter settings for optimizing the output to see without restrictions on view limit
jupyter.textOutputLimit = 0

In [ ]:
# Reading JSON file with credentials to access Reddit API Wrapper
creds = 'client_secrets.json'

with open(creds) as f:
    data_creds = json.load(f)

data_creds

In [ ]:
# Creating an instance from Reddit class to interact with Reddit API

reddit = praw.Reddit(
    client_id=data_creds['client_id'],
    client_secret=data_creds['client_secret'],
    user_agent=data_creds['user_agent'],
    redirect_uri=data_creds['redirect_uri'],
    refresh_token=data_creds['refresh_token']
)


In [ ]:
# Choosing a subreddit to analyse
subr = 'wallstreetbets'

# Obtaining a instance of this class for subreddit
subreddit = reddit.subreddit(subr)

dir(subreddit)

In [ ]:
from praw.models import MoreComments

# Obtaining top 10 hottest posts of today on chosen subreddit
top_10_hottest = subreddit.hot(limit=10)
boolean = True

# Obtaining for each post 20 recent comments.
for post in top_10_hottest:
    print('\033[1m Title: {0} \033[0m \n'.format(post.title))

    for num, comm in enumerate(post.comments):
        if num < 20:
            if isinstance(comm, MoreComments):
                print('\033[1m {0}) MoreComments:\033[0m {1}'.format(str(num), str(comm.comments()[0:100])))
                continue
            # Max 100 characters per comment
            print('\033[1m {0}) Comment:\033[0m {1}'.format(str(num), comm.body[0:100]))
        else:
            break
    print('\n')


In [ ]:
# Obtaining 100 newest comments of today on chosen subreddit, not on specific submission

subr2 = 'VIAC'
subr = 'wallstreetbets'

subreddit2 = reddit.subreddit(subr2)

subreddit_wsb = reddit.subreddit(subr)

# for Obtaining new post titles
subreddit2_new = subreddit2.new()

# obtaining newest comments for the subreddit regardless of the submission
subr_comments_new = subreddit_wsb.comments()

# for obtaining newly posted comments regardless of the submission
subreddit2_comments = subreddit2.comments()



#for num, comment in enumerate(subreddit2_comments):
#    print(""" 
#        {0}) Comment: {1}
#        URL: {2}
#        Author: {3}
#        """.format(str(num), comment.body, comment.link_url, comment.author))

for num, comment in enumerate(subr_comments_new):
    print(""" 
        {0}) Comment: {1}
        URL: {2}
        Author: {3}
        """.format(str(num), comment.body, comment.link_url, comment.author))



In [ ]:
# Code to search for specific patterns via Regex library
url_submission = 'https://www.reddit.com/r/wallstreetbets/comments/ob59hl/what_are_your_moves_tomorrow_july_01_2021/'
submission_post = reddit.submission(url=url_submission)

pattern = '\wiacom|[Vv][Ii][Aa][Cc]'
pattern2 = '[Aa][Mm][Cc]'

# Converting a pattern into a regular expression object
reg_exp2 = re.compile(pattern)

# array to store matched comments that mentioned ViacomCBS ticker/name
mentioned = []

for num, comment in enumerate(submission_post.comments):
    if isinstance(comment, MoreComments):
        for comm_in in comment.comments():
            print('\033[1m MoreComments:\033[0m {0}'.format(comm_in.body))
            #print('\033[1m {0}) MoreComments:\033[0m {1}'.format(str(num), comment.replace_more_comments().comment))
        continue
    result = reg_exp2.search(comment.body)
    if result:
        mentioned.append(comment.body)
    print('\033[1m {0}) Comment:\033[0m {1}'.format(str(num), comment.body[0:100]))


In [ ]:
mentioned

In [ ]:
last_update = mentioned.copy()
last_update

In [ ]:
len(submission_post.comments)